In [138]:
import numpy as np
from nptdms import TdmsFile
import os

path_base_folder = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Patients ePatch data"
output_base_folder = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data"

patient_path_1 = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Patients ePatch data\Patient 1\recording 2\Patient 1_2.tdms"

Patients_to_use = ["Patient 3", "Patient 5", "Patient 6", "Patient 8", "Patient 10", "Patient 14", "Patient 15", "Patient 16", "Patient 21", "Patient 23", "Patient 27", "Patient 28", "Patient 29", "Patient 31", "Patient 34", "Patient 37", "Patient 39", "Patient 40", "Patient 41", "Patient 42"]

In [139]:
def correct_tdms_start_time(tdms_file_path, output_path="copied_file.tdms"):
    from nptdms import TdmsFile, TdmsWriter, RootObject
    import numpy as np
    import re
    from datetime import datetime

    original_file = TdmsFile(tdms_file_path)
    original_groups = original_file.groups()
    original_channels = [chan for group in original_groups for chan in group.channels()]
    original_file.properties["Corrected_by: "] = "Kasper Viken Jensen"
    starttime_string = original_file.properties["name"]

    # Extract start datetime information from the start_time string
    datetime_matches = re.findall(r'\d{2}_\d{2}_\d{4} kl_?\d{6}', starttime_string)
    if len(datetime_matches) > 0:
        start_dt_str = datetime_matches[0].replace(' kl_', ' ').replace(' kl', ' ')
        start_datetime = datetime.strptime(start_dt_str, '%d_%m_%Y %H%M%S')
    if len(datetime_matches) == 0:
        datetime_matches = re.findall(r'\d{2}-\d{2}-\d{4} kl_?\d{6}', starttime_string)
        
    else:
        start_datetime = None

    # Correct the start time in channel properties if found
    if start_datetime is not None:
        original_channels[0].properties["wf_start_time"] = np.datetime64(start_datetime)

    # Write to new TDMS file
    with TdmsWriter(output_path) as copied_file:
        root_object = RootObject(original_file.properties)
        copied_file.write_segment([root_object] + original_groups + original_channels)
    return output_path


In [142]:
final_patient_folders = []
folders = [folder for folder in os.listdir(path_base_folder) if folder in Patients_to_use]

for folder in folders:
    folder_path = os.path.join(path_base_folder, folder)
    output_path = os.path.join(output_base_folder, folder)
    os.makedirs(output_path, exist_ok=True)

    for root, dirs, files in os.walk(folder_path):
        if not dirs:  # Deepest folder
            final_patient_folders.append(root)

print("Final patient folders:", final_patient_folders)

for folder1 in final_patient_folders:
    files_in_folder = os.listdir(folder1)
    tdms_files = [f for f in files_in_folder if f.endswith('.tdms')]
    if tdms_files:
        tdms_file_path = os.path.join(folder1, tdms_files[0])
        output = os.path.join(output_base_folder, os.path.basename(folder1), tdms_files[0])
        x = correct_tdms_start_time(tdms_file_path, output)
        print("TDMS file path:", tdms_file_path)
        print("Output path:", output)
        print("Corrected file:", x)
        print("TDMS files:", tdms_files)


Final patient folders: ['D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatch data from Aarhus to Lausanne\\Patients ePatch data\\Patient 10\\recording 1', 'D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatch data from Aarhus to Lausanne\\Patients ePatch data\\Patient 14\\recording 1', 'D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatch data from Aarhus to Lausanne\\Patients ePatch data\\Patient 14\\recording 2', 'D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatch data from Aarhus to Lausanne\\Patients ePatch data\\Patient 15\\recording 1', 'D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatch data from Aarhus to Lausanne\\Patients ePatch data\\Patient 15\\recording 2', 'D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatch data from Aarhus to Lausanne\\Patients ePatch data\\Patient 16\\recording 1', 'D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatch data from Aarhus to Lausanne\\Patients ePatch data\\Patient 21\\recording 1', 'D:\\ML algoritme tl anfaldsdetektion vha HRV\\ePatc

UnboundLocalError: cannot access local variable 'start_datetime' where it is not associated with a value

In [118]:

patient_path = r"D:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Patients ePatch data\Patient 10\recording 1\Patient 10_1.tdms"

# Remove leading/trailing whitespace and ensure correct joining
patient_path_clean = patient_path.strip().lstrip("\\/")
tdms_file_path = os.path.join(path_base_folder, patient_path_clean)

# print(path_base_folder)
# print(patient_path)
# print(patient_path_1)
# print(tdms_file_path)

if not os.path.exists(tdms_file_path):
    raise FileNotFoundError(f"Path does not exist: {tdms_file_path}")

In [131]:
td = TdmsFile(patient_path_clean)
ch = td.groups

print(ch)

<bound method TdmsFile.groups of <nptdms.tdms.TdmsFile object at 0x0000027981C964D0>>


In [132]:

ch = td.groups()[0].channels()[0]

# Data
x = ch.data.astype(float)

print(ch.properties)

# Print the first few lines of raw TDMS file metadata
print("TDMS File Properties:")
print(td.properties)

tdms_prop = td.properties

group = td.groups()

print("\nTDMS Groups:")
for group in td.groups():
    print(f"Group: {group.name}")
    print("Properties:", group.properties)
    print("Channels:", [ch.name for ch in group.channels()])
    for channel in group.channels():
        print(f"  Channel: {channel.name}")
        print("  Properties:", channel.properties)
        print("  First 5 data points:", channel.data[:5])
        print()
# Samplingfrekvens
inc = ch.properties.get("wf_increment", None)

# Starttid
start_time = ch.properties.get("wf_start_time", None)

OrderedDict([('wf_start_time', numpy.datetime64('2017-06-20T11:44:43.000000')), ('wf_start_offset', 0.0), ('wf_increment', 0.001953125), ('wf_samples', 132710144), ('DigitalMaximum', 32767), ('DigitalMinimum', -32768), ('NI_ChannelName', 'EKG'), ('NI_ExpIsRelativeTime', False), ('wf_time_pref', 'absolute'), ('NI_ExpStartTimeStamp', numpy.datetime64('2017-06-20T11:44:43.000000')), ('NI_ExpTimeStamp', numpy.datetime64('2017-06-20T11:44:43.000000')), ('NI_ExpXDimension', 't'), ('wf_xname', 'Time'), ('wf_xunit_string', 's'), ('NI_UnitDescription', 'uV'), ('unit_string', 'uV'), ('PhysicalMaximum', 90298.0), ('PhysicalMinimum', -90298.0), ('PreFilter', ''), ('TransducerType', ''), ('ValsPerRec', 128)])
TDMS File Properties:
OrderedDict([('name', 'Sidsel Borg Nielsen 20062017 kl_134443 __23062017_kl_130000.tdms'), ('author', 'Jesper'), ('description', ''), ('registertxt1', 'Written by National Instruments LabVIEW')])

TDMS Groups:
Group: Untitled
Properties: OrderedDict([('wf_xcolumns', 'None

In [ ]:
# print(group)
# print(td)

<TdmsGroup with path /'Untitled'>


In [ ]:
# prop = ch.properties

# print(prop["wf_start_time"])

# print(x)

2016-10-12T09:05:02.000000
[ -70.27081712   -1.37785916   37.2021973  ... -125.38518349 -128.14090181
 -125.38518349]


In [ ]:
# start_time = td.properties["name"]
# print(start_time)

Thue Bundgaard 12_10_2016 kl_110502__14_10_2016_ kl_080000.tdms


In [ ]:
# import re
# from datetime import datetime

# # Extract start and end datetime information from the start_time string
# datetime_matches = re.findall(r'\d{2}_\d{2}_\d{4} kl_?\d{6}', start_time)

# datetime_end = re.findall(r'__\d{2}_\d{2}_\d{4}_ kl_?\d{6}', start_time)

# # Parse start datetime
# if len(datetime_matches) > 0:
#     start_dt_str = datetime_matches[0].replace(' kl_', ' ').replace(' kl', ' ')
#     start_datetime = datetime.strptime(start_dt_str, '%d_%m_%Y %H%M%S')
#     print("Parsed start datetime:", start_datetime)
# else:
#     start_datetime = None
#     print("No valid start datetime found.")

# # Parse end datetime
# if len(datetime_end) > 0:
#     end_dt_str = datetime_end[0].replace(' kl_', ' ').replace(' kl', ' ')
#     end_datetime = datetime.strptime(end_dt_str, '__%d_%m_%Y_ %H%M%S')
#     print("Parsed end datetime:", end_datetime)
# else:
#     end_datetime = None
#     print("No valid end datetime found.")


Parsed start datetime: 2016-10-12 11:05:02
Parsed end datetime: 2016-10-14 08:00:00


In [ ]:
# import numpy as np

# # Update start_time with the correct date, hour, and minute from start_datetime

# corrected_start_time = np.datetime64(start_datetime)
# print("Corrected start_time:", corrected_start_time)

# print("Before correction:", td.groups()[0].channels()[0].properties["wf_start_time"])
# td.groups()[0].channels()[0].properties["wf_start_time"] = corrected_start_time
# print("After correction:", td.groups()[0].channels()[0].properties["wf_start_time"])

# td.properties["Corrected_by: "] = "Kasper Viken Jensen"
# print(td.properties["Corrected_by: "])

# # prop["wf_start_time"] = corrected_start_time
# # print(prop["wf_start_time"])

Corrected start_time: 2016-10-12T11:05:02.000000
Before correction: 2016-10-12T11:05:02.000000
After correction: 2016-10-12T11:05:02.000000
Kasper Viken Jensen


In [ ]:
# tdms_prop["Corrected_by: "] = "Kasper Viken Jensen"

# print(tdms_prop["Corrected_by: "])
# print(tdms_prop)

Kasper Viken Jensen
OrderedDict([('name', 'Thue Bundgaard 12_10_2016 kl_110502__14_10_2016_ kl_080000.tdms'), ('author', 'Jesper'), ('description', ''), ('registertxt1', 'Written by National Instruments LabVIEW'), ('Corrected_by: ', 'Kasper Viken Jensen')])
